# Linear Regression Experiments

Interactive exploration of linear regression concepts.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Import our from-scratch implementation
import sys
sys.path.insert(0, '.')
from scratch import LinearRegressionScratch, RidgeRegressionScratch

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Simple Linear Regression

Visualize fitting a line to 2D data.

In [ ]:
# Generate 1D data
n = 50
X = np.random.uniform(0, 10, n).reshape(-1, 1)
y = 2 + 3 * X.ravel() + np.random.randn(n) * 2

# Fit model
model = LinearRegressionScratch(method='normal_equation')
model.fit(X, y)

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.7, label='Data')

# Regression line
X_line = np.linspace(0, 10, 100).reshape(-1, 1)
y_line = model.predict(X_line)
plt.plot(X_line, y_line, 'r-', linewidth=2, label=f'ŷ = {model.bias_:.2f} + {model.weights_[0]:.2f}x')

plt.xlabel('X')
plt.ylabel('y')
plt.title('Simple Linear Regression')
plt.legend()
plt.show()

print(f"True: y = 2 + 3x")
print(f"Learned: y = {model.bias_:.2f} + {model.weights_[0]:.2f}x")
print(f"R² = {model.score(X, y):.4f}")

## 2. Normal Equation vs. Gradient Descent

Compare the two optimization methods.

In [ ]:
# Generate data
n = 100
X = np.random.randn(n, 3)
true_w = np.array([2, -1, 0.5])
y = X @ true_w + 1 + np.random.randn(n) * 0.2

# Normal equation
model_ne = LinearRegressionScratch(method='normal_equation')
model_ne.fit(X, y)

# Gradient descent
model_gd = LinearRegressionScratch(method='gradient_descent')
model_gd.fit(X, y, learning_rate=0.1, n_iterations=500)

# Compare
print(f"True weights:     {true_w}")
print(f"Normal equation:  {model_ne.weights_}")
print(f"Gradient descent: {model_gd.weights_}")
print(f"\nTrue bias: 1.0")
print(f"Normal equation bias:  {model_ne.bias_:.4f}")
print(f"Gradient descent bias: {model_gd.bias_:.4f}")

In [ ]:
# Plot gradient descent convergence
plt.figure(figsize=(10, 4))
plt.plot(model_gd.loss_history_)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Gradient Descent Convergence')
plt.show()

print(f"Initial loss: {model_gd.loss_history_[0]:.4f}")
print(f"Final loss: {model_gd.loss_history_[-1]:.4f}")

## 3. Effect of Learning Rate

Explore how learning rate affects convergence.

In [ ]:
learning_rates = [0.001, 0.01, 0.1, 0.5]

plt.figure(figsize=(12, 4))

for lr in learning_rates:
    model = LinearRegressionScratch(method='gradient_descent')
    model.fit(X, y, learning_rate=lr, n_iterations=200)
    plt.plot(model.loss_history_, label=f'lr={lr}')

plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Effect of Learning Rate')
plt.legend()
plt.ylim(0, 2)
plt.show()

## 4. Overfitting and Regularization

Demonstrate overfitting with polynomial features and fix with regularization.

In [ ]:
# Generate non-linear data
n = 20
X = np.sort(np.random.uniform(0, 1, n))
y = np.sin(2 * np.pi * X) + np.random.randn(n) * 0.3

# Create polynomial features
def poly_features(x, degree):
    return np.column_stack([x**i for i in range(1, degree + 1)])

degrees = [1, 3, 9, 15]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

X_plot = np.linspace(0, 1, 100)

for ax, degree in zip(axes, degrees):
    X_poly = poly_features(X, degree)
    X_plot_poly = poly_features(X_plot, degree)
    
    model = LinearRegression()
    model.fit(X_poly, y)
    y_plot = model.predict(X_plot_poly)
    
    ax.scatter(X, y, alpha=0.7)
    ax.plot(X_plot, y_plot, 'r-')
    ax.plot(X_plot, np.sin(2 * np.pi * X_plot), 'g--', alpha=0.5, label='True')
    ax.set_title(f'Degree {degree}')
    ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

In [ ]:
# Ridge regularization on high-degree polynomial
degree = 15
X_poly = poly_features(X, degree)
X_plot_poly = poly_features(X_plot, degree)

alphas = [0, 0.001, 0.01, 0.1]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, alpha in zip(axes, alphas):
    if alpha == 0:
        model = LinearRegression()
    else:
        model = Ridge(alpha=alpha)
    
    model.fit(X_poly, y)
    y_plot = model.predict(X_plot_poly)
    
    ax.scatter(X, y, alpha=0.7)
    ax.plot(X_plot, y_plot, 'r-')
    ax.plot(X_plot, np.sin(2 * np.pi * X_plot), 'g--', alpha=0.5)
    ax.set_title(f'Ridge α={alpha}')
    ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

## 5. Ridge vs Lasso: Sparsity

Compare coefficient shrinkage patterns.

In [ ]:
# Generate data with some irrelevant features
n = 100
X = np.random.randn(n, 10)
# Only first 3 features are relevant
true_coef = np.array([3, -2, 1, 0, 0, 0, 0, 0, 0, 0])
y = X @ true_coef + np.random.randn(n) * 0.5

# Compare coefficients
linear = LinearRegression().fit(X, y)
ridge = Ridge(alpha=1.0).fit(X, y)
lasso = Lasso(alpha=0.1).fit(X, y)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

x_pos = np.arange(10)

axes[0].bar(x_pos, linear.coef_)
axes[0].set_title('Linear Regression')
axes[0].set_xlabel('Feature')
axes[0].set_ylabel('Coefficient')

axes[1].bar(x_pos, ridge.coef_)
axes[1].set_title('Ridge (α=1.0)')
axes[1].set_xlabel('Feature')

axes[2].bar(x_pos, lasso.coef_)
axes[2].set_title('Lasso (α=0.1)')
axes[2].set_xlabel('Feature')

plt.tight_layout()
plt.show()

print(f"True coefficients: {true_coef}")
print(f"Lasso non-zero:    {np.sum(lasso.coef_ != 0)} / 10")

## 6. Bias-Variance Tradeoff

Visualize the tradeoff with different regularization strengths.

In [ ]:
from sklearn.model_selection import cross_val_score

# Generate data
n = 100
X = np.random.randn(n, 5)
y = X @ np.array([1, 2, 3, 4, 5]) + np.random.randn(n) * 2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

alphas = np.logspace(-4, 4, 50)
train_scores = []
test_scores = []

for alpha in alphas:
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    train_scores.append(model.score(X_train, y_train))
    test_scores.append(model.score(X_test, y_test))

plt.figure(figsize=(10, 5))
plt.semilogx(alphas, train_scores, label='Train R²')
plt.semilogx(alphas, test_scores, label='Test R²')
plt.xlabel('Regularization (α)')
plt.ylabel('R² Score')
plt.title('Bias-Variance Tradeoff')
plt.legend()
plt.axvline(alphas[np.argmax(test_scores)], color='r', linestyle='--', alpha=0.5)
plt.show()

best_alpha = alphas[np.argmax(test_scores)]
print(f"Best α: {best_alpha:.4f}")
print(f"Best test R²: {max(test_scores):.4f}")

## 7. Residual Analysis

Diagnose model fit using residual plots.

In [ ]:
# Generate well-specified data
n = 100
X = np.random.randn(n, 1)
y = 2 + 3*X.ravel() + np.random.randn(n) * 0.5

model = LinearRegression().fit(X, y)
y_pred = model.predict(X)
residuals = y - y_pred

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Residuals vs Predicted
axes[0].scatter(y_pred, residuals, alpha=0.6)
axes[0].axhline(0, color='r', linestyle='--')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Residual')
axes[0].set_title('Residuals vs Predicted')

# Histogram of residuals
axes[1].hist(residuals, bins=20, edgecolor='black')
axes[1].set_xlabel('Residual')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution')

# Q-Q plot
from scipy import stats
stats.probplot(residuals, dist='norm', plot=axes[2])
axes[2].set_title('Q-Q Plot')

plt.tight_layout()
plt.show()

## 8. Comparison with sklearn

Verify our implementation matches sklearn.

In [ ]:
# Generate data
n = 100
X = np.random.randn(n, 5)
y = X @ np.array([1, 2, 3, 4, 5]) + 2 + np.random.randn(n) * 0.5

# Our implementation
model_scratch = LinearRegressionScratch(method='normal_equation')
model_scratch.fit(X, y)

# sklearn
model_sklearn = LinearRegression()
model_sklearn.fit(X, y)

print("Weights comparison:")
print(f"Scratch:  {model_scratch.weights_}")
print(f"sklearn:  {model_sklearn.coef_}")
print(f"\nBias comparison:")
print(f"Scratch:  {model_scratch.bias_:.6f}")
print(f"sklearn:  {model_sklearn.intercept_:.6f}")
print(f"\nR² comparison:")
print(f"Scratch:  {model_scratch.score(X, y):.6f}")
print(f"sklearn:  {model_sklearn.score(X, y):.6f}")
print(f"\nMax weight difference: {np.max(np.abs(model_scratch.weights_ - model_sklearn.coef_)):.10f}")

## 9. Real World Example: California Housing

Apply linear regression to a real dataset.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Load data
housing = fetch_california_housing()
X, y = housing.data, housing.target
feature_names = housing.feature_names

print(f"Dataset shape: {X.shape}")
print(f"Features: {feature_names}")

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train models
linear = LinearRegression().fit(X_train_scaled, y_train)
ridge = Ridge(alpha=1.0).fit(X_train_scaled, y_train)

print(f"\nLinear Regression - Test R²: {linear.score(X_test_scaled, y_test):.4f}")
print(f"Ridge Regression - Test R²: {ridge.score(X_test_scaled, y_test):.4f}")

In [ ]:
# Feature importance
importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': linear.coef_,
    'Abs Coefficient': np.abs(linear.coef_)
}).sort_values('Abs Coefficient', ascending=False)

plt.figure(figsize=(10, 5))
plt.barh(importance['Feature'], importance['Coefficient'])
plt.xlabel('Coefficient')
plt.title('Feature Importance (Linear Regression)')
plt.tight_layout()
plt.show()

## Key Takeaways

1. **Normal equation** gives exact solution but is expensive for high dimensions
2. **Gradient descent** scales better but requires tuning learning rate
3. **Regularization** prevents overfitting by shrinking weights
4. **Lasso** produces sparse solutions (feature selection)
5. **Ridge** shrinks all weights but none to zero
6. **Residual analysis** helps diagnose model quality